# Pandasの使い方

Pandasの使い方で身につける狙いは、書いたコードを『読んで説明できる』状態にすることです。


## 観察 1: DataFrameを作って集計する

表形式データを扱うときは DataFrame を中心に考えます。まず最小例で列操作を体験します。


In [ ]:
scores = [58, 72, 81, 67, 90]
labels = ['A', 'B', 'C', 'D', 'E']
paired = list(zip(labels, scores))
print('paired =', paired)
try:
    import pandas as pd
except ModuleNotFoundError:
    print('pandas is not installed')
else:
    df = pd.DataFrame({'name': ['A', 'B', 'C'], 'score': [72, 88, 95]})
    print(df)
    print('mean score =', df['score'].mean())

列ごとに意味を持った処理を書けるので、前処理の可読性と再利用性が上がります。

この節では、変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 2: 条件で分ける

次に、条件分岐で情報を分類します。分類は機械学習の前処理でも頻出なので、早い段階で手を慣らしておきます。


In [ ]:
passed = []
for name, score in paired:
    if score >= 70:
        passed.append(name)
print('passed =', passed)

このコードは単純ですが、評価基準を変更したときの影響範囲が読みやすい形になっています。基準値を一箇所に集約すると保守しやすくなります。

この節では、変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 計算の対応表

1. 厳密な式より、入力 -> 変換 -> 出力の流れを言語化することが中心です。


## 観察 3: 関数で再利用する

同じ処理を繰り返すときは、関数で意味を名前にします。名前付けは可読性の中心で、後から読む自分を助ける実装でもあります。


In [ ]:
def normalize(xs):
    lo, hi = min(xs), max(xs)
    span = hi - lo if hi != lo else 1
    return [(x - lo) / span for x in xs]
print('normalized =', normalize(scores))

ここで大切なのは、ゼロ除算のような例外条件を先に潰すことです。初学者は正常系だけを考えがちですが、異常系を一行でも書くとコードの寿命が伸びます。

この節では、変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 4: 辞書で意味を持たせる

配列だけでは意味が曖昧になるとき、辞書でキーを明示します。キーは設計者の意図を運ぶラベルです。


In [ ]:
records = [{'name': n, 'score': s} for n, s in paired]
top = max(records, key=lambda r: r['score'])
avg = sum(r['score'] for r in records) / len(records)
print('top =', top)
print('avg =', round(avg, 2))

辞書形式にすると、列の追加や削除に強くなります。機械学習の特徴量を増減するときにも同じ考え方が使えます。

この節では、変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 観察 5: 小さな検証を自動化する

最後に、期待する性質を `assert` で固定します。これはテストの最小形で、理解の確認にも使えます。


In [ ]:
normalized = normalize(scores)
assert len(normalized) == len(scores)
assert min(normalized) >= 0.0
assert max(normalized) <= 1.0
print('checks passed')

コードを読むだけでなく、実行して仮説を確認する流れを作ると学習効率が上がります。以後の章でもこの確認ループを維持してください。

この節では、変数 が入出力のどこを決めるかを中心に読める状態になれば十分です。


## 要点整理

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 型が想定と違っていても気づかず処理が進んでしまう
2. 条件分岐の境界値を検証しない
3. 同じロジックを複数箇所にコピペして保守不能になる
